In [ ]:
# Create sensing grid

from mintpy import iono_tec

In [ ]:
import os
import site
from pathlib import Path
from isce2_topsapp import iono_proc
from isce.applications import topsApp
# Update PATH with ISCE2 applications
isce_app_path = Path(f"{site.getsitepackages()[0]}" "/isce/applications/")
os.environ["PATH"] += ":" + str(isce_app_path)

tops_app_cmd = f"{isce_app_path}/topsApp.py"
os.chdir('/u/trappist-r0/govorcin/ARIA-tools/IONO/isce/docker_A_B')
#Load input file
topsapp = topsApp.TopsInSAR(name="topsApp", cmdline=['topsApp.xml'])
topsapp.configure()

#Load pickle objekt from 1 step before ion: fine resampale
topsapp.loadPickleObj('/u/trappist-r0/govorcin/ARIA-tools/IONO/isce/docker_A_B/PICKLE/fineresamp')

In [ ]:
ionParam = iono_proc.runIon.setup(topsapp)


In [ ]:
iono_proc.ionSwathBySwath(topsapp, ionParam)



In [ ]:
%matplotlib inline
import os
import datetime as dt

import geopandas as gpd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, colorbar, ticker, colors
from scipy import interpolate, stats
from shapely.geometry import Point

from mintpy import iono_tec
from mintpy.objects import sensor, ionex
from mintpy.simulation import iono
from mintpy.utils import ptime, readfile, writefile
plt.rcParams.update({'font.size': 12})

#proj_dir = os.path.expanduser('.')
#work_dir = os.path.join(proj_dir, 'TEC')
#os.chdir(work_dir)
#print('Go to directory:', work_dir)

# aux info
tec_dir = os.path.expanduser('~/data/aux/IONEX')
#tf_file = os.path.join(work_dir, 'tframe_left_look.gpkg')

inc_angle = 42
inc_angle_iono = iono.incidence_angle_ground2iono(inc_angle)

In [ ]:
tec_files = []
for year in range(2014, 2023):
    date_list = ptime.get_date_range(f'{year}0101', f'{year}1231')
    tec_files += iono_tec.download_ionex_files(date_list, tec_dir, sol_code='jpl')

In [ ]:

from numpy.typing import NDArray
from isce.components.isceobj.Constants import SPEED_OF_LIGHT
from isce.applications import topsApp

def get_sensingtime_grid(self, lats, lons, hgts) -> NDArray:

    #Get frames and orbit
    swathList = self._insar.getValidSwathList(topsapp.swaths)
    frames = []
    for swath in swathList:
        referenceProduct = self._insar.loadProduct( os.path.join(self._insar.fineCoregDirname, 'IW{0}.xml'.format(swath)))
        frames.append(referenceProduct)

    # Load orbits
    orb = self._insar.getMergedOrbit(frames)

    # Create sensing grid array
    length, width = lat_ds.RasterYSize, lat_ds.RasterXSize 

    sensing_grid = np.zeros((length, width, 2), dtype=np.float64)

    for iy in range(0,length,1):
        for ix in range(0,width,1):
            if lats[iy,ix] !=0:
                pixel_sensing_az_rg = orb.geo2rdr(llh=(lats[iy,ix],
                                                       lons[iy,ix],
                                                       hgts[iy,ix]))
                
                sensing_grid[iy,ix,0] = pixel_sensing_az_rg[0].timestamp()
                sensing_grid[iy,ix,1] = (2*pixel_sensing_az_rg[1]) / SPEED_OF_LIGHT
             
    return sensing_grid

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

def get_nc_groups(filename_nc : str):
    from osgeo import gdal
    #open nc file
    ds = gdal.Open(filename_nc)
    metadata = ds.GetMetadata()

    # Get Groups
    groups_df = pd.DataFrame(columns=['LAYER','PATH'])
    for layer in ds.GetSubDatasets():
        layer_df = pd.DataFrame(data={'LAYER':[layer[0].split(':')[-1].split('/')[-1]],
                                       'PATH':[layer[0]]})
        groups_df = pd.concat([groups_df, layer_df], ignore_index=True)
    #close
    ds = None

    return groups_df, metadata

def read_nc_group(group_path:str):
    from osgeo import gdal
    # group_path : 'NETCDF:"$path/file.nc":group_layer
    # example: 'NETCDF:"$PATH/S1-GUNW-D-R-021-tops-20230210_20230129-033529-00035E_00034N-PP-9780-v2_0_6.nc":/science/grids/data/unwrappedPhase'

    # open group
    ds = gdal.Open(group_path, gdal.GA_ReadOnly)

    # Get group/raster array
    data =  ds.ReadAsArray()
    nodata = ds.GetRasterBand(1).GetNoDataValue()
    metadata = ds.GetMetadata()

    # Get raster geographical information
    trans = ds.GetGeoTransform()
    xsize = ds.RasterXSize
    ysize = ds.RasterYSize
    snwe = [trans[3] + ysize * trans[5], trans[3],
              trans[0], trans[0] + xsize * trans[1]]
    lon_spacing = trans[1]
    lat_spacing = trans[5]
    
    projection = ds.GetProjection()

    # wrap raster info in dict
    raster_dict = {'NODATA' : nodata,
                   'LENGTH' : ysize,
                   'WIDTH'  : xsize,
                   'SNWE'   : snwe,
                   'LON_SPACING' : lon_spacing,
                   'LAT_SPACING' : lat_spacing,
                   'PROJECTION' : projection}

    # close
    ds = None

    return data, raster_dict, metadata

# get image extent from attr['snwe'] for plotting
def snwe_to_extent(snwe):
    extent = [snwe[2], snwe[3], snwe[0], snwe[1]]
    
    return extent

In [ ]:
import os
os.chdir('/u/trappist-r0/govorcin/ARIA-tools/IONO/isce/docker_test2')

In [ ]:
os.listdir('S1-GUNW-A-R-064-tops-20211120_20211108-014951-00119W_00030N-PP-aed8-v2_0_6')

In [ ]:
get_nc_groups('S1-GUNW-A-R-064-tops-20211120_20211108-014951-00119W_00030N-PP-aed8-v2_0_6/S1-GUNW-A-R-064-tops-20211120_20211108-014951-00119W_00030N-PP-aed8-v2_0_6.nc')

In [ ]:
from osgeo import gdal
ds = gdal.Open('S1-GUNW-A-R-064-tops-20211120_20211108-014951-00119W_00030N-PP-aed8-v2_0_6/S1-GUNW-A-R-064-tops-20211120_20211108-014951-00119W_00030N-PP-aed8-v2_0_6.nc')

In [ ]:
for layer in ds.GetSubDatasets():
    print(layer)

In [ ]:
metadata = ds.GetMetadata()
metadata

In [ ]:
import h5py

In [ ]:
file = h5py.File('merged/metadata.h5', 'r')

In [ ]:
dateime(midtight) + timedelta(second=10000)

In [ ]:
data = file['cube']
print(data.keys())

In [ ]:
data['lats'], data['lons'], data['heights']

In [ ]:
data['lats'].shape[0]

In [ ]:
data['lats'][:]

In [ ]:
sensing_grid = np.zeros((data['lats'].shape[0],
                         data['lons'].shape[0], 
                         data['heights'].shape[0]))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(sensing_grid[:,:,0])

In [ ]:
nz_idx = np.nonzero(data['lats'][:])

In [ ]:
os.path.join(os.getcwd(),'PICKLE/geocode')

In [ ]:
import h5py
import os
from isce.components.isceobj.Constants import SPEED_OF_LIGHT
from datetime import timedelta
from isce.applications import topsApp

file = h5py.File('merged/metadata.h5', 'r')
data = file['cube']


#Load input file
topsapp = topsApp.TopsInSAR(name="topsApp", cmdline=['topsApp.xml'])
topsapp.configure()

#Load pickle object
topsapp.loadPickleObj(os.path.join(os.getcwd(),'PICKLE/geocode'))

#Get frames and orbit
swathList = topsapp._insar.getValidSwathList(topsapp.swaths)
frames = []
for swath in swathList:
    referenceProduct = topsapp._insar.loadProduct(os.path.join(topsapp._insar.fineCoregDirname, 'IW{0}.xml'.format(swath)))
    frames.append(referenceProduct)

# Load orbits
orb = topsapp._insar.getMergedOrbit(frames)

sensing_grid = np.zeros((data['lats'].shape[0],
                         data['lons'].shape[0], 
                         data['heights'].shape[0]))

for iy, lat in enumerate(data['lats'][:]):
    for ix, lon in enumerate(data['lons'][:]):
        for iz, hgt in enumerate(data['heights'][:]):
            pixel_sensing_az_rg = orb.geo2rdr(llh=(lat,lon,hgt))                                              # approx, should alwys start from 1 swath
            sensing = pixel_sensing_az_rg[0] + timedelta(seconds=(pixel_sensing_az_rg[1]) / SPEED_OF_LIGHT) # azimuth_sensing_time +( ix * PRF + range/speed of light)
            sensing_grid[iy,ix,iz]  = sensing.timestamp()

In [ ]:
(lat,lon,hgt) 1/PRF

In [ ]:
orb.geo2rdr(llh=(30,-115,1050))[0] - orb.geo2rdr(llh=(30,-115,1000))[0] 

In [ ]:
pixel_sensing_az_rg[1] / SPEED_OF_LIGHT

In [ ]:
frames[1].bursts.burst1.firstValidSample * (1/frames[1].bursts.burst1.prf)

In [ ]:
frames[1].bursts.burst1.lastValidSample * (1/frames[1].bursts.burst1.prf) + pixel_sensing_az_rg[1] / SPEED_OF_LIGHT

In [ ]:
np.broadcast_to(data['lats'][:], (72,47))

In [ ]:
from datetime import datetime

In [ ]:
datetime.fromtimestamp(sensing_grid[:,:,3])

In [ ]:
from datetime import datetime
[datetime.fromtimestamp(i) for i in sensing_grid[0,:,3]]

In [ ]:
plt.imshow(sensing_grid[:,:,])
plt.colorbar()

In [ ]:
datetime.fromtimestamp()

In [ ]:
lats = data['lats'][:]
lons = data['lons'][:]
heights = data['heights'][:]

# Reshape the arrays to create a 3D grid
ny, nx, nz = (lats.shape[0], 
              lons.shape[0], 
              heights.shape[0])
lats = lats.reshape(ny, nx, 1)
lons = lons.reshape(ny, nx, 1)
heights = heights.reshape(1, 1, nz)


pixels_sensing_az_rg = orb.geo2rdr(llh=(lats, lons, heights))

In [ ]:
coords = np.stack((data['lats'][:], data['lons'][:], data['heights'][:]), axis=-1)